# Inspect data


In [ ]:
import sys
import os
import numpy as np
import plotly.graph_objects as go
import json
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath("../../Leaderboard/"))

from leaderboard.autoagents.autonomous_agent import AutonomousAgent
from leaderboard.agents.geometric_map import GeometricMap
from leaderboard.utils.constants import Constants

from lac.mapping.mapper import Mapper, interpolate_heights, grid_to_points
from lac.mapping.map_utils import get_geometric_score
from lac.utils.plotting import plot_poses, plot_3d_points, plot_surface
from lac.util import load_data, pos_rpy_to_pose

%load_ext autoreload
%autoreload 2

### Load data


In [ ]:
data_path = "../../output/LocalizationAgent/map1_preset0_4m_spiral"
# data_path = "../../output/NavAgent/map1_preset9_seed2_nofid"
# data_path = "../../output/Old/LocalizationAgent_spiral_norocks"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")
ekf_result = np.load(f"{data_path}/ekf_result.npz")

### Inspect poses


In [ ]:
ekf_poses = []
for state in ekf_result["xhat_smooth"]:
    ekf_poses.append(pos_rpy_to_pose(state[0:3], state[6:9]))

In [ ]:
fig = plot_poses(poses[::10], color="blue", no_axes=True, name="Ground Truth")
fig = plot_poses(ekf_poses[::10], fig=fig, color="orange", no_axes=True, name="EKF")
fig.show()

### Inspect odometry


In [ ]:
json_data = json.load(open(f"{data_path}/data_log.json"))
N = len(json_data["frames"])
wheel_odoms = np.zeros((N, 2))
for i, frame in enumerate(json_data["frames"]):
    wheel_odoms[i] = [frame["linear_speed"], frame["angular_speed"]]

### Inspect maps


In [ ]:
# mapper = Mapper()
constants = Constants(qualifier=True, testing=False)
g_map = GeometricMap(constants)
mapper = Mapper(g_map)
mapper.wheel_contact_update(ekf_poses)
m1 = g_map.get_map_array()
mapper.finalize_heights()
m2 = g_map.get_map_array()

In [ ]:
height_array = g_map.get_map_array()
height_array_interpolated = interpolate_heights(height_array)

In [ ]:
fig = plot_3d_points(grid_to_points(height_array_interpolated), color="blue")
fig = plot_3d_points(grid_to_points(height_array), fig=fig, color="red")
fig.update_layout(height=700, width=1200, scene_aspectmode="data")
fig.show()

In [ ]:
get_geometric_score(height_array, height_array_interpolated)

In [ ]:
ground_truth_map = np.load(f"{data_path}/Moon_Map_02_11_rep0.dat", allow_pickle=True)
agent_map = np.load(f"{data_path}/Moon_Map_02_11_rep0_agent.dat", allow_pickle=True)

In [ ]:
plot_surface(ground_truth_map)

In [ ]:
plot_surface(agent_map)

In [ ]:
get_geometric_score(ground_truth_map, agent_map)

In [ ]:
get_geometric_score(ground_truth_map, height_array)

In [ ]:
get_geometric_score(ground_truth_map, height_array_interpolated)

In [ ]:
ground_truth_map_points = ground_truth_map[:, :, :3].reshape(-1, 3)
agent_map_points = agent_map[:, :, :3].reshape(-1, 3)

fig = plot_3d_points(ground_truth_map_points, color="blue", name="Ground Truth")
fig = plot_3d_points(agent_map_points, fig=fig, color="red", name="Agent Map")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

# Inspect rock points


In [ ]:
data_path = "../../output/nav_agent"
rock_points = np.load(f"{data_path}/rock_points.npy")

In [ ]:
fig = plot_3d_points(rock_points)
fig.update_layout(height=700, width=1200, scene_aspectmode="data")
fig.show()

In [ ]:
import os

cams_geoms = json.load(open(os.path.expanduser("~/LunarAutonomyChallenge/docs/geometry.json")))[
    "rover"
]["cameras"]
cams_geoms["FrontLeft"]